# 1. Dữ liệu sơ khai

In [41]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import functools
from concurrent.futures import ThreadPoolExecutor
import time

In [42]:
# Load environment variables in a file called .env

#load_dotenv(override=True)
#os.environ['OPENAI_API_KEY'] = os.getenv('DEEPSEEK_API_KEY')
#openai = OpenAI()


load_dotenv(override=True)  
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

openai = OpenAI(
    api_key=deepseek_api_key,
    base_url="https://api.deepseek.com/v1"  
)

MODEL = "deepseek-chat"
db_name = "vector_db"

In [43]:
# Performance improvement: Use caching for context loading
@functools.lru_cache(maxsize=None)
def load_context():
    """Load and cache context data to avoid repeated file I/O"""
    context = {}
    
    # Load employees with threading for faster I/O
    def load_employee_files():
        employee_context = {}
        employees = glob.glob("knowledge-base/employees/*")
        
        def load_single_employee(employee):
            name = employee.split(' ')[-1][:-3]
            try:
                with open(employee, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {employee}: {e}")
                return name, ""
        
        # Use ThreadPoolExecutor for concurrent file loading
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_employee, employees)
            employee_context.update(dict(results))
        
        return employee_context
    
    # Load schools with threading
    def load_school_files():
        school_context = {}
        schools = glob.glob("knowledge-base/schools/*")
        
        def load_single_school(school):
            name = school.split(os.sep)[-1][:-3]
            try:
                with open(school, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {school}: {e}")
                return name, ""
        
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_school, schools)
            school_context.update(dict(results))
        
        return school_context
    
    # Load both types concurrently
    with ThreadPoolExecutor(max_workers=2) as executor:
        employee_future = executor.submit(load_employee_files)
        school_future = executor.submit(load_school_files)
        
        context.update(employee_future.result())
        context.update(school_future.result())
    
    return context

In [44]:
# Load context once at startup
import os
print("Thư mục hiện tại:", os.getcwd())
context = load_context()
print(f"Loaded context with {len(context)} documents: {list(context.keys())}")

Thư mục hiện tại: d:\LLM\llmprojects\chatbot\agents
Loaded context with 11 documents: ['Kim', 'Lee', 'Lan', 'Jae', 'Park', 'Ewha Womans University', 'KAIST', 'Korea University', 'POSTECH', 'Seoul National University', 'Yonsei University']


In [45]:
context["Lan"]

'# Bà Nguyễn Thị Lan - Giám Đốc Điều Hành\n\n## Thông Tin Cá Nhân\n\n**Họ và tên**: Nguyễn Thị Lan\n**Chức vụ**: Giám Đốc Điều Hành (CEO)  \n**Tuổi**: 42  \n**Quốc tịch**: Việt Nam  \n**Ngôn ngữ**: Tiếng Việt (bản ngữ), Tiếng Hàn (Thành thạo - TOPIK 6), Tiếng Anh (Thành thạo - IELTS 8.0)  \n\n## Liên Hệ\n\n📞 **Điện thoại**: 0901-111-222  \n📧 **Email**: lan.nguyen@koreastudyvn.com  \n📧 **Email cá nhân**: ceo@koreastudyvn.com  \n🏢 **Văn phòng**: Tầng 12, Lotte Center, Hà Nội  \n\n## Học Vấn\n\n### Bằng Cấp Chính\n- **2010**: Thạc sĩ Quản trị Kinh doanh (MBA) - Đại học Yonsei, Seoul, Hàn Quốc\n- **2005**: Cử nhân Quan hệ Quốc tế - Đại học Ngoại thương, Hà Nội, Việt Nam\n\n### Chứng Chỉ Bổ Sung\n- **2018**: Chứng chỉ Tư vấn Giáo dục Quốc tế - ICEF (International Consultants for Education and Fairs)\n- **2016**: Chứng chỉ Quản lý Dự án - PMP (Project Management Professional)\n- **2012**: Chứng chỉ Tiếng Hàn TOPIK Level 6\n\n## Kinh Nghiệm Làm Việc\n\n### 2018 - Hiện tại: Giám Đốc Điều Hành\

In [46]:
system_message = (
    "Bạn là một chuyên gia tư vấn du học Hàn Quốc tại trung tâm Korea Study. "
    "Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến trung tâm, nhân viên, trường học, và thông tin visa một cách ngắn gọn và chính xác. "
    "Nếu bạn không biết câu trả lời, hãy nói rõ rằng bạn không biết. "
    "Tuyệt đối không bịa ra thông tin nếu không có ngữ cảnh liên quan được cung cấp."
)

In [47]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context  

In [48]:
def add_context(message):
    """Add relevant context to message"""
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nNhững thông tin sau có thể hữu ích cho việc trả lời câu hỏi này:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [49]:
def chat(message, history):
    """Optimized chat function with better error handling"""
    try:
        messages = [{"role": "system", "content": system_message}] + history
        message = add_context(message)
        messages.append({"role": "user", "content": message})

        stream = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            stream=True,
            max_tokens=1000,  # Limit response length for faster generation
            temperature=0.7
        )

        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                yield response
    except Exception as e:
        yield f"Xin lỗi, đã có lỗi xảy ra: {str(e)}"

In [16]:
# Launch first version
print("Launching keyword-based RAG chatbot...")
view = gr.ChatInterface(chat, type="messages").launch()

Launching keyword-based RAG chatbot...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# 2. Dữ liệu Vector

In [50]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [51]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase
import glob
import os
folders = glob.glob("knowledge-base/*")

#text_loader_kwargs = {'encoding': 'utf-8'}
# Nếu dòng trên không hoạt động, người dùng Windows có thể dùng dòng dưới thay thế
text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print("Total documents loaded:", len(documents))


Total documents loaded: 17


In [52]:
documents[1]

Document(metadata={'source': 'knowledge-base\\company\\overview.md', 'doc_type': 'company'}, page_content='# Tổng Quan Công Ty - Korea Study Consultant Center\n\n## Thông Tin Cơ Bản\n\n**Tên công ty**: Korea Study Consultant Center  \n**Tên tiếng Hàn**: 한국 유학 상담 센터  \n**Tên viết tắt**: KSCC  \n**Năm thành lập**: 2018  \n**Loại hình**: Công ty tư vấn giáo dục quốc tế  \n**Mã số thuế**: 0123456789  \n\n## Thông Tin Liên Hệ\n\n### Trụ Sở Chính - Hà Nội\n📍 **Địa chỉ**: Tầng 12, Tòa nhà Lotte Center, 54 Liễu Giai, Ba Đình, Hà Nội  \n📞 **Điện thoại**: (024) 3935-2468  \n📧 **Email**: hanoi@koreastudyvn.com  \n🕒 **Giờ làm việc**: 8:00 - 18:00 (Thứ 2 - Chủ nhật)  \n\n### Chi Nhánh TP.HCM\n📍 **Địa chỉ**: Tầng 8, Tòa nhà Bitexco, 2 Hải Triều, Quận 1, TP.HCM  \n📞 **Điện thoại**: (028) 3824-1357  \n📧 **Email**: hcm@koreastudyvn.com  \n🕒 **Giờ làm việc**: 8:00 - 18:00 (Thứ 2 - Chủ nhật)  \n\n### Văn Phòng Đại Diện Seoul\n📍 **Địa chỉ**: #1203, Gangnam Finance Center, Gangnam-gu, Seoul, South Korea  \

In [53]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Slightly smaller chunks for better retrieval
    chunk_overlap=100,  # Reduced overlap for performance
    separators=["\n\n", "\n", ". ", " ", ""]  # Better separation
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

Created 155 chunks


In [54]:
chunks[9]

Document(metadata={'source': 'knowledge-base\\company\\overview.md', 'doc_type': 'company'}, page_content='## Trường Đối Tác Chính\n\n### Top Universities\n- Seoul National University (서울대학교)\n- Yonsei University (연세대학교)  \n- Korea University (고려대학교)\n- KAIST (한국과학기술원)\n- POSTECH (포항공과대학교)\n\n### Private Universities\n- Hanyang University (한양대학교)\n- Kyung Hee University (경희대학교)\n- Ewha Womans University (이화여자대학교)\n- Sogang University (서강대학교)\n- Sungkyunkwan University (성균관대학교)\n\n### Language Institutes\n- Yonsei Korean Language Institute\n- Seoul National University LEI\n- Sogang Korean Language Program\n- Ewha Language Center\n\n## Khu Vực Phục Vụ\n\n### Việt Nam\n- **Miền Bắc**: Hà Nội, Hải Phòng, Nam Định, Thái Bình\n- **Miền Trung**: Đà Nẵng, Huế, Vinh, Quy Nhon\n- **Miền Nam**: TP.HCM, Cần Thơ, Vũng Tàu, Đồng Nai')

In [55]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_types)}")

Các loại tài liệu đã tìm thấy: visas, schools, company, employees


In [27]:
for chunk in chunks:
    if 'Nguyễn Thị Lan' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='### 4. Hiệu Quả
- Rút ngắn thời gian xử lý hồ sơ
- Chi phí hợp lý, minh bạch
- Cam kết hoàn tiền nếu không đậu visa

## Thành Tựu Nổi Bật

### Số Liệu Thống Kê
- **2,000+** học sinh được hỗ trợ thành công
- **98.5%** tỷ lệ đậu visa du học
- **50+** trường đại học đối tác
- **95%** học sinh hài lòng với dịch vụ

### Giải Thưởng
- **2022**: Trung tâm tư vấn du học uy tín nhất Việt Nam
- **2021**: Đại lý chính thức xuất sắc của Đại học Yonsei
- **2020**: Top 3 công ty tư vấn du học Hàn Quốc tại Việt Nam

## Đội Ngũ Lãnh Đạo

### Giám Đốc Điều Hành
**Bà Nguyễn Thị Lan** - 15 năm kinh nghiệm trong lĩnh vực giáo dục quốc tế, từng học tập và làm việc tại Hàn Quốc 8 năm.' metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}
_________
page_content='### Thành Viên Của
- **Hiệp hội Tư vấn Du học Việt Nam (VIECA)**
- **Liên minh Giáo dục Việt Nam - Hàn Quốc**
- **Mạng lưới Đối tác Giáo dục Quốc tế (IEPN)**

### Đối Tác Chính Thức
- **Bộ Giáo dục Hàn Quốc (M

In [56]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [57]:
# Đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

#embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ HuggingFace (thay vì OpenAI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [58]:

MODEL = "gpt-4o-mini"

# Đặt tên cho database vector (có thể tùy chọn)
db_name = "vector_db"

# Kiểm tra nếu database Chroma đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [59]:
# Tạo vector store bằng Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,              # Danh sách các đoạn văn bản đã chia nhỏ
    embedding=embeddings,          # Hàm embedding (ví dụ: OpenAI hoặc HuggingFace)
    persist_directory=db_name      # Thư mục lưu trữ database
)
# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


Vectorstore created with 155 documents


In [60]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [61]:
sample_embedding

array([-5.58394603e-02,  4.88829017e-02,  3.94378528e-02, -5.21505922e-02,
       -4.73595820e-02,  6.04551192e-03,  6.68961480e-02,  3.65742035e-02,
        4.03285138e-02,  1.48874531e-02,  1.15573399e-01, -4.77703884e-02,
       -1.07764611e-02, -4.73897792e-02, -1.96826868e-02, -3.49632986e-02,
       -4.65893596e-02,  1.07359933e-02, -2.19003130e-02, -9.36346576e-02,
       -2.50792298e-02,  9.96900536e-03,  1.84770604e-03,  1.98029075e-02,
       -2.14104969e-02, -8.33652541e-03, -1.06111486e-02,  3.39315943e-02,
        5.35235666e-02, -7.21928012e-03, -1.94507167e-02,  1.51503772e-01,
       -2.40047444e-02, -1.92200132e-02,  2.50265338e-02,  1.09274108e-02,
       -2.03468408e-02, -1.25908703e-02,  3.48537005e-02, -1.17554925e-02,
       -5.45321517e-02, -7.40574151e-02,  5.93916737e-02, -8.16977099e-02,
        3.85917202e-02,  9.44460742e-03, -7.49501362e-02, -1.97852068e-02,
       -1.06640393e-02,  2.60305516e-02, -4.29825783e-02,  4.46056165e-02,
       -7.03462283e-04,  

In [62]:
# Lấy toàn bộ vector, tài liệu và metadata từ collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Đưa embedding vào mảng numpy
vectors = np.array(result['embeddings'])

# Lưu lại văn bản
documents = result['documents']

# Trích loại tài liệu từ metadata (giả sử có 'doc_type')
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]

# Gán màu sắc tùy theo loại tài liệu
colors = [['blue', 'green', 'red', 'orange'][['company', 'employees', 'visas', 'schools'].index(t)] for t in doc_types]


In [63]:
# Con người chúng ta dễ hình dung mọi thứ trong không gian 2D hơn!
# Giảm số chiều của vector xuống 2D bằng t-SNE
# (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 2D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="browser")

In [64]:
from langchain.memory import ConversationBufferWindowMemory  
from langchain.chains import ConversationalRetrievalChain

In [65]:
# Tạo mô hình Chat với OpenAI
#llm = ChatOpenAI(
#   temperature=0.7, 
#    model_name=MODEL,
#)
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(
    temperature=0.7,
    model="deepseek-chat",
    base_url="https://api.deepseek.com/v1",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)
# Thiết lập bộ nhớ hội thoại

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    return_messages=True
)

# Retriever từ vectorstore (FAISS, Chroma, v.v.)
retriever = vectorstore.as_retriever()

# Kết nối RAG pipeline
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

In [66]:
# Test query with performance monitoring
import time
def test_query_performance():
    """Test query with timing"""
    query = "Bạn có thể mô tả ngắn gọn về Korea study center ?"
    start_time = time.time()
    result = conversation_chain.invoke({"question": query})
    end_time = time.time()
    
    print(f"Query processed in {end_time - start_time:.2f} seconds")
    print("Answer:", result["answer"])
    if "source_documents" in result:
        print(f"Used {len(result['source_documents'])} source documents")

In [67]:
test_query_performance()

Query processed in 16.54 seconds
Answer: Korea Study Consultant Center là trung tâm tư vấn du học Hàn Quốc được thành lập năm 2018, chuyên hỗ trợ học sinh Việt Nam tiếp cận hệ thống giáo dục Hàn Quốc. 

**Đặc điểm nổi bật:**
- Đội ngũ chuyên gia giàu kinh nghiệm về giáo dục Hàn Quốc
- Đã hỗ trợ hơn 2,000 học sinh Việt Nam du học thành công
- Dịch vụ toàn diện từ tư vấn trường, hồ sơ đến định hướng nghề nghiệp
- Tầm nhìn trở thành trung tâm tư vấn du học Hàn Quốc hàng đầu tại Việt Nam

Trung tâm hoạt động với sứ mệnh kết nối thế hệ trẻ Việt Nam với các cơ hội giáo dục chất lượng cao tại Hàn Quốc.


In [68]:
# set up a new conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [69]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [70]:
# And in Gradio:
import gradio as gr
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


3. Ý tưởng tìm hiểu fine-tuning

In [41]:
# Hãy cùng tìm hiểu xem điều gì được gửi phía sau hậu trường

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.7,
    model="deepseek-chat",
    base_url="https://api.deepseek.com/v1",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)

query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Các trường/khoa nổi bật

### Trường Kinh doanh (School of Business)
- **Quản trị kinh doanh**
- **Tài chính và ngân hàng**
- **Marketing quốc tế**
- **Kinh doanh quốc tế**
- **MBA programs**

### Trường Y học
- **Y khoa (6 năm)**
- **Điều dưỡng**
- **Y học cộng đồng**
- **Bệnh viện đại học Ewha Mokdong**

### Trường Kỹ thuật
- **Khoa học máy tính**
- **Kỹ thuật điện và điện tử**
- **Kỹ thuật hóa học và vật liệu**
- **Kỹ thuật xây dựng**
- **Cybersecurity**

### Trường Nghệ thuật
- **Âm nhạc**
- **Mỹ thuật**
- **Thiết kế**
- **Khoa học sân khấu**
- **Art therapy**

### Trường Nhân văn
- **Ngôn ngữ và văn học Hàn Quốc**
- **Ngôn ngữ học ứng dụng*

In [43]:
# create a new Chat with OpenAI
llm = ChatOpenAI(
    temperature=0.7,
    model="deepseek-chat",
    base_url="https://api.deepseek.com/v1",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

# set up the conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)

query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Các trường/khoa nổi bật

### Trường Kinh doanh (School of Business)
- **Quản trị kinh doanh**
- **Tài chính và ngân hàng**
- **Marketing quốc tế**
- **Kinh doanh quốc tế**
- **MBA programs**

### Trường Y học
- **Y khoa (6 năm)**
- **Điều dưỡng**
- **Y học cộng đồng**
- **Bệnh viện đại học Ewha Mokdong**

### Trường Kỹ thuật
- **Khoa học máy tính**
- **Kỹ thuật điện và điện tử**
- **Kỹ thuật hóa học và vật liệu**
- **Kỹ thuật xây dựng**
- **Cybersecurity**

### Trường Nghệ thuật
- **Âm nhạc**
- **Mỹ thuật**
- **Thiết kế**
- **Khoa học sân khấu**
- **Art therapy**

### Trường Nhân văn
- **Ngôn ngữ và văn học Hàn Quốc**
- **Ngôn ngữ học ứng dụng*

In [44]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]


In [45]:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


# 3 Improve RAG Ensemble Hybrid Retrieval.

In [46]:
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever
from typing import List, Dict

# 1. Fix keyword retriever to search content, not just title
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def get_relevant_documents(self, query: str) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(Document(page_content=content, metadata={"source": title}))
        return relevant_docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

# 2. Use keyword + properly configured vector retriever
keyword_retriever = KeywordRetriever(context_dict=context)
vector_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 30}
)

# 3. Ensemble
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 4. Conversation chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)


C:\Users\Hoa.Ngo\AppData\Local\Temp\ipykernel_5576\2883139772.py:6: DeprecationWarning:

Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`

C:\Users\Hoa.Ngo\AppData\Local\Temp\ipykernel_5576\2883139772.py:6: DeprecationWarning:

Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`



NameError: name 'context' is not defined

In [ ]:
query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)


In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

# Notification Tool

In [7]:
# some new imports
import requests

In [5]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    
push("Timi!!")

In [9]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [10]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [11]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [12]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [13]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [16]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results